# Compute the readout fidelity of all qubits

In [3]:
import numpy as np
from qibo.gates import M, X
from qibo.models import Circuit
from qiboconnection.api import API
from qiboconnection.connection import ConnectionConfiguration

api = API(ConnectionConfiguration(username="qat", api_key="meow"))

api.select_device_id(9)

We first generate and queue the circuits we need to execute:

In [4]:
NSHOTS = 2000  # 1000

In [5]:
circuits = []
qubits = np.arange(5)
for qubit in qubits:
    circuit0 = Circuit(5)
    circuit0.add(M(qubit))

    circuit1 = Circuit(5)
    circuit1.add(X(qubit))
    circuit1.add(M(qubit))

    circuits.extend([circuit0, circuit1])

result_id = api.execute(circuits, nshots=NSHOTS)[0]

print(f"The id of the executed job is: {result_id}")

The id of the executed job is: 9608


Once the jobs have been executed, we can use the cell below to compute the readout fidelities:

In [7]:
results = api.get_result(result_id)

Your job with id 9608 is still pending. Job queue position: 3


In [11]:
for qubit_idx in qubits:
    # We take the pair of results that correspond to the specific qubit
    results0 = results[qubit_idx * 2]
    results1 = results[qubit_idx * 2 + 1]

    clf_0 = (1 - results0["probabilities"]["0"] + results0["probabilities"]["1"]) / 2
    clf_1 = (1 - results1["probabilities"]["0"] + results1["probabilities"]["1"]) / 2
    fid_avg = (1 - clf_0 + clf_1) / 2
    print(f"### QUBIT: {qubit_idx} ###")
    print(f"Ground state: {results0}")
    print(f"Excited state: {results1}")
    print("Fidelity", fid_avg)

### QUBIT: 0 ###
Ground state: {'probabilities': {'0': 0.867, '1': 0.133}}
Excited state: {'probabilities': {'0': 0.143, '1': 0.857}}
Fidelity 0.862
### QUBIT: 1 ###
Ground state: {'probabilities': {'0': 0.871, '1': 0.129}}
Excited state: {'probabilities': {'0': 0.2465, '1': 0.7535}}
Fidelity 0.81225
### QUBIT: 2 ###
Ground state: {'probabilities': {'0': 0.883, '1': 0.117}}
Excited state: {'probabilities': {'0': 0.1595, '1': 0.8405}}
Fidelity 0.86175
### QUBIT: 3 ###
Ground state: {'probabilities': {'0': 0.88, '1': 0.12}}
Excited state: {'probabilities': {'0': 0.1495, '1': 0.8505}}
Fidelity 0.8652500000000001
### QUBIT: 4 ###
Ground state: {'probabilities': {'0': 0.916, '1': 0.084}}
Excited state: {'probabilities': {'0': 0.1365, '1': 0.8635}}
Fidelity 0.88975
